In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
import keras

Using TensorFlow backend.


# Data

In [2]:
import tensorflow.keras.utils 
import tensorflow as tf

In [3]:
data_train = np.load("../input/chinese-char-recognition-smmo19/train-1.npy", allow_pickle=True)
data_test = np.load('../input/chinese-char-recognition-smmo19/test.npy', allow_pickle=True)
#np.random.shuffle(data_train)
for i in range(2, 5):
    t = np.load(f"../input/chinese-char-recognition-smmo19/train-{i}.npy", allow_pickle=True)
    data_train = np.concatenate([data_train, t])
    
IMG_SIZE = 80
DATASET_SIZE = len(data_train)
batch_size = 128
val_size = 0.1

tokens = np.unique([v[1] for v in data_train])
id_to_char = {id:char for id, char in enumerate(tokens)}
char_to_id = {char:id for id, char in enumerate(tokens)}

print(data_train.shape)
data_train = data_train.tolist()

def train_gen():
    for img, label in data_train[int(len(data_train)*val_size):]:
        img = img[..., None] # [batch, w, h, channels]
        yield img, char_to_id[label] #char_to_vector[]

def val_gen():
    for img, label in data_train[:int(len(data_train)*val_size)]:
        img = img[..., None] # [batch, w, h, channels]
        yield img, char_to_id[label] 

def test_gen():
    for img in data_test:
        img = img[..., None] # [batch, w, h, channels]
        yield img 

(332987, 2)


In [4]:
def augmentation(image):
    number = np.random.random_sample()
    if 0.5 <= number :
        image = flip(image)
        image = zoom(image)
        image = color(image)
    return image

def flip(x: tf.Tensor) -> tf.Tensor:
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)
    return x

def color(x: tf.Tensor) -> tf.Tensor:
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x

def zoom(x: tf.Tensor) -> tf.Tensor:
    # Generate 20 crop settings, ranging from a 1% to 20% crop.
    scales = list(np.arange(0.8, 1.0, 0.01))
    boxes = np.zeros((len(scales), 4))
    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(img):
        crops = tf.image.crop_and_resize([img], boxes=boxes, box_indices=np.zeros(len(scales)), crop_size=(32, 32))
        return crops[tf.random.uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]

    choice = tf.random.uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)
    # Only apply cropping 50% of the time
    return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))

In [5]:
from tensorflow.keras.applications.xception import preprocess_input

In [6]:
def preprocess_train(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = augmentation(image)
    image = tf.cast(image, tf.float32)/ 127.5 - 1
    image = preprocess_input(image)
    label = tf.one_hot(label, 1000)
    return image, label

def preprocess_val(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 127.5 - 1
    image = preprocess_input(image)
    label = tf.one_hot(label, 1000)
    return image, label

def preprocess_test(image):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32)/ 127.5 - 1
    image = preprocess_input(image)
    return image

In [7]:
ds_train = tf.data.Dataset.from_generator(train_gen,
                                          output_types=(tf.float32, tf.int32), 
                                          output_shapes=((None,None,1), ())).map(preprocess_train, tf.data.experimental.AUTOTUNE).prefetch(-1).shuffle(1000).batch(batch_size).repeat()

ds_val = tf.data.Dataset.from_generator(val_gen,
                                          output_types=(tf.float32, tf.int32), 
                                          output_shapes=((None,None,1), ())).map(preprocess_val, tf.data.experimental.AUTOTUNE).prefetch(-1).shuffle(1000).batch(batch_size).repeat()

ds_test = tf.data.Dataset.from_generator(test_gen,
                                          output_types=(tf.float32), 
                                          output_shapes=((None,None,1))
                                          ).map(preprocess_test, tf.data.experimental.AUTOTUNE).prefetch(-1).batch(batch_size)

# Приступим к обучению

In [8]:
initial_epochs = 40
steps_per_epoch = round(len(data_train))//(batch_size*10)
validation_steps = round(len(data_train)*val_size)//(batch_size*10)

IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 1)
INIT_LR = 5e-3

In [9]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
#from keras.applications import VGG19
# from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

In [10]:
base_model = tensorflow.keras.applications.xception.Xception(weights=None,
                                               input_shape=IMG_SHAPE, 
                                               classes=1000)

In [11]:
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

base_model.compile(
        loss= 'categorical_crossentropy',
        optimizer= tf.keras.optimizers.Adam(lr=INIT_LR),
        metrics=['accuracy'])

In [12]:
import datetime

In [13]:
print(f'>> Begin training at {datetime.datetime.now()}')
base_model.fit(ds_train,epochs=15,
                  validation_data = ds_val,
                   steps_per_epoch=2500,
                   validation_steps=200, verbose = 1, 
                    callbacks = [tf.keras.callbacks.ModelCheckpoint('weights1.{epoch:02d}.hdf5', best_only=True, monitor='accuracy', period = 10),
                                 tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
                  ])
print(f'>> Training finished at {datetime.datetime.now()}')

>> Begin training at 2019-12-23 16:54:00.977815
Train for 2500 steps, validate for 200 steps
Epoch 1/15
2500/2500 [==============================] - 486s 194ms/step - loss: 1.6626 - accuracy: 0.6562 - val_loss: 64.1313 - val_accuracy: 0.0015
Epoch 2/15
2500/2500 [==============================] - 478s 191ms/step - loss: 0.1527 - accuracy: 0.9555 - val_loss: 79.9647 - val_accuracy: 9.3750e-04
Epoch 3/15
2500/2500 [==============================] - 478s 191ms/step - loss: 0.0831 - accuracy: 0.9749 - val_loss: 79.4770 - val_accuracy: 9.3750e-04
Epoch 4/15
2500/2500 [==============================] - 478s 191ms/step - loss: 0.0498 - accuracy: 0.9848 - val_loss: 33.5277 - val_accuracy: 9.3750e-04
Epoch 5/15
2500/2500 [==============================] - 477s 191ms/step - loss: 0.0334 - accuracy: 0.9897 - val_loss: 6962.7984 - val_accuracy: 8.9844e-04
Epoch 6/15
2500/2500 [==============================] - 479s 192ms/step - loss: 0.0023 - accuracy: 0.9993 - val_loss: 35.4932 - val_accuracy: 9.

In [14]:
print(f'>> Begin training at {datetime.datetime.now()}')
base_model.fit(ds_train,epochs=20,
                  validation_data = ds_val,
                   steps_per_epoch=2000,
                   validation_steps=100, verbose = 1, 
                    callbacks = [tf.keras.callbacks.ModelCheckpoint('weights2.{epoch:02d}.hdf5', best_only=True, monitor='accuracy', period = 5),
                                 tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
                  ])
print(f'>> Training finished at {datetime.datetime.now()}')

>> Begin training at 2019-12-23 18:53:51.291480
Train for 2000 steps, validate for 100 steps
Epoch 1/20
2000/2000 [==============================] - 381s 190ms/step - loss: 0.0587 - accuracy: 0.9831 - val_loss: 50.1197 - val_accuracy: 0.0010
Epoch 2/20
2000/2000 [==============================] - 380s 190ms/step - loss: 0.0217 - accuracy: 0.9931 - val_loss: 136.5429 - val_accuracy: 0.0015
Epoch 3/20
2000/2000 [==============================] - 381s 190ms/step - loss: 0.0170 - accuracy: 0.9948 - val_loss: 39.3473 - val_accuracy: 0.0020
Epoch 4/20
2000/2000 [==============================] - 379s 190ms/step - loss: 0.0141 - accuracy: 0.9954 - val_loss: 21.6367 - val_accuracy: 0.0021
Epoch 5/20
2000/2000 [==============================] - 380s 190ms/step - loss: 0.0105 - accuracy: 0.9967 - val_loss: 32.6523 - val_accuracy: 0.0011
Epoch 6/20
2000/2000 [==============================] - 379s 190ms/step - loss: 0.0080 - accuracy: 0.9975 - val_loss: 31.4287 - val_accuracy: 0.0013
Epoch 7/20
2

In [15]:
#base_model.evaluate(ds_val, verbose= 1, steps = 10000)

In [16]:
print(f'>> Begin prediction at {datetime.datetime.now()}')
result = base_model.predict(ds_test, verbose=1)
predictions_without_cat = result.argmax(-1) #[np.argmax(predict) for predict in result] 
predictions = [id_to_char[predict] for predict in predictions_without_cat]
print(f'>> Finished prediction at {datetime.datetime.now()}')

>> Begin prediction at 2019-12-23 21:00:40.266660
    651/Unknown - 32s 49ms/step>> Finished prediction at 2019-12-23 21:01:12.478192


In [17]:
df = pd.read_csv('../input/chinese-char-recognition-smmo19/random_labels.csv')
df['Category'] = predictions
df.to_csv('submission.csv', index=False)
np.savetxt("submision_please_do_it.csv", predictions, delimiter=";")

In [18]:
# import tensorflow.keras.applications
# from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
#from tensorflow.keras.applications.xception import preprocess_input
#from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input